In [1]:
#%% 
import xarray as xr
from PyCO2SYS import sys
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
import pandas as pd

from cProfile import label
from json import load
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from PyCO2SYS import sys
from scipy.io import loadmat
import warnings
warnings.filterwarnings("ignore")

import warnings

# 3rd party libraries
import statsmodels.api as sm
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import AxesGrid
import cartopy.feature as cfeature

from statsmodels.tools.eval_measures import rmse
import statsmodels.api as sm
from scipy.interpolate import interp2d
from scipy.interpolate import griddata
import scipy

projection = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(map_projection=projection))

In [12]:
#%%
yr_min = 1993
time_min = np.datetime64('1993-01-01')
time_max = np.datetime64('2021-12-31')
lon_min = -100
lon_max = -40
lat_min = 10
lat_max = 65

In [14]:
file = '/Volumes/Crucial_4T/carbon/ReCAD-NAC-pCO2/ReCAD-NAC-pCO2_v1.nc'
ds_product = xr.open_dataset(file)

time = ds_product.time.values
lon = ds_product.lon.values
lat = ds_product.lat.values

idxlon = (lon>=lon_min) & (lon<=lon_max)
idxlat = (lat>=lat_min) & (lat<=lat_max)

lon = ds_product.lon.values[idxlon]
lat = ds_product.lat.values[idxlat]

fco2_socat = ds_product.fco2_socat[:,idxlat,idxlon].values
fco2_product = ds_product.fco2_product[:,idxlat,idxlon].values
pco2_product = ds_product.pco2_product[:,idxlat,idxlon].values

file_err = '/Volumes/Crucial_4T/carbon/ReCAD-NAC-pCO2/ReCAD-NAC-pCO2_err_v1.nc'
ds_product_err = xr.open_dataset(file_err)
fco2_product_err = ds_product_err.err[:,idxlat,idxlon].values

In [17]:
ds = xr.Dataset(
    data_vars=dict(
        # fco2_product_re =(["time", "lat", "lon"], fco2_product_calib),
        fco2_socat =(["time", "lat", "lon"], fco2_socat),
        fco2_product =(["time", "lat", "lon"], fco2_product),
        pco2_product =(["time", "lat", "lon"], pco2_product),
        fco2_product_err =(["time", "lat", "lon"], fco2_product_err),

    ),
    coords=dict(
        time=(["time"], time),
        lat=(["lat"], lat),
        lon=(["lon"], lon),
    ),
    attrs=dict(description="ReCAD-pCO2, fCO2 is derived from SOCAT fCO2 directly, pCO2 is converted from fCO2 using SST from OISST."),
)
ds.to_netcdf('/Volumes/Crucial_4T/carbon/ReCAD-NAC-pCO2/ReCAD-NAACOM-pCO2_v1.nc')

In [18]:
ds

<xarray.Dataset>
Dimensions:           (time: 348, lat: 220, lon: 240)
Coordinates:
  * time              (time) datetime64[ns] 1993-01-16T12:00:00 ... 2021-12-1...
  * lat               (lat) float64 10.12 10.38 10.62 ... 64.38 64.62 64.88
  * lon               (lon) float64 -99.88 -99.62 -99.38 ... -40.38 -40.12
Data variables:
    fco2_socat        (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    fco2_product      (time, lat, lon) float64 nan nan nan nan ... nan nan nan
    pco2_product      (time, lat, lon) float64 nan nan nan nan ... nan nan nan
    fco2_product_err  (time, lat, lon) float64 nan nan nan nan ... nan nan nan
Attributes:
    description:  ReCAD-pCO2, fCO2 is derived from SOCAT fCO2 directly, pCO2 ...